In [1]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

from tensorflow.compat.v1.keras.backend import get_session
tf.compat.v1.disable_v2_behavior()
#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset8.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[1,2,17,18,19])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)
ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=20, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                             

Instructions for updating:
non-resource variables are not supported in the long term
Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.121477
                      0.210609
                      0.363499
                      0.263091
                      1.090112
                      0.084009
                      0.030310
                      0.424584
                      0.114606
                      0.023002
                      0.315954
                      0.098218
                      0.016568
                      0.198494
                      0.301346
                      0.047236
                      0.465671
                      0.176065
                      0.309285
                      0.222463
 DT - feature_importance_vals
                     0.004383
                     0.063247
                     0.039435
                     0.008451
                     0.158970
                     0.000000
                     0.014184
                     0.025313
                     0.001566
                     0.022096
                   

keras is no longer supported, please use tf.keras instead.


 NN - feature_importance_vals
                     0.012872
                     0.051876
                     0.103038
                     0.075571
                     0.044795
                     0.014302
                     0.037607
                     0.025596
                     0.020633
                     0.042350
                     0.015902
                     0.013839
                     0.015880
                     0.022844
                     0.032197
                     0.013364
                     0.050606
                     0.103626
                     0.115842
                     0.103881
Iteration 2


...

 XGB - feature_importance_vals
                      0.135973
                      0.267540
                      0.340773
                      0.202678
                      1.040858
                      0.084239
                      0.092416
                      0.410883
                      0.087750
                      0.034186
                      0.303421
                      0.101105
                      0.014962
                      0.182400
                      0.293825
                      0.075861
                      0.495921
                      0.157909
                      0.161535
                      0.223150
 DT - feature_importance_vals
                     0.004948
                     0.044281
                     0.032935
                     0.000000
                     0.137295
                     0.005378
                     0.063468
                     0.022187
                     0.009455
                     0.031337
                   

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.124039
                      0.268192
                      0.344478
                      0.218038
                      1.086507
                      0.098074
                      0.012078
                      0.440062
                      0.097600
                      0.022819
                      0.299217
                      0.099824
                      0.019645
                      0.220843
                      0.299805
                      0.030933
                      0.461830
                      0.161775
                      0.137066
                      0.187459
 DT - feature_importance_vals
                     0.004179
                     0.044301
                     0.028898
                     0.000000
                     0.165156
                     0.007927
                     0.010198
                     0.020210
                     0.010261
                     0.033360
                   

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.137691
                      0.267191
                      0.333687
                      0.194226
                      1.105857
                      0.083793
                      0.008005
                      0.428108
                      0.099357
                      0.008383
                      0.288082
                      0.100862
                      0.016640
                      0.179476
                      0.300934
                      0.069363
                      0.486651
                      0.192966
                      0.173144
                      0.205932
 DT - feature_importance_vals
                     0.000286
                     0.047397
                     0.031394
                     0.000000
                     0.032005
                     0.007711
                     0.157309
                     0.041267
                     0.000000
                     0.011833
                   